 # `AVDOS-VR` - Virtual Reality Affective Video Database with Physiological Signals

Notebook containing the scripts to verify the data and summarize the sessions for the AVDOS-VR dataset.

It analyses the events (.json) and physiological responses (.csv) per participant, and produces a set of files: 

1) EMG Signal quality per participant measured at the fit assessmeent stage shortly before video segment commenced.
2) Plots visualising EMG signal quality for duration of the each video segments (1-5)
3) CSV file including average mean/arousal ratings and number of ratings for each video
4) CSV file including various duration metrics

In [1]:
from pathlib import Path
import sys,os
this_path = None
try:    # WORKS WITH .py
    this_path = str(os.path.dirname(os.path.abspath(__file__)))
except: # WORKS WITH .ipynb
    this_path = str(Path().absolute())+"/" 
print("File Path:", this_path)

# Add the level up to the file path so it recognizes the scripts inside `avdos-vr`
sys.path.append(os.path.join(this_path, ".."))

File Path: D:\AVDOS-VR\notebooks/


In [2]:
import os
import numpy as np
import pandas as pd

import avdosvr
from avdosvr.verify_data import verify_data
from avdosvr.plots.plots import plot_sm_ppg, plot_fit_state
from avdosvr.utils.classes import Participant
from avdosvr.utils.files_handler import generate_complete_path

from avdosvr.analysis.average_ratings import calculate_average_arousal_valence_ratings, get_average_av_ratings_columns
from avdosvr.analysis.durations import calculate_good_signal_quality_duration, calculate_signal_quality_check_duration, get_durations_columns
from avdosvr.analysis.skip_participants import skip_participant

import avdosvr.hrv.HRV_analysis_cloud as HRV

# Hide warning about pandas Slicing
pd.options.mode.chained_assignment = None


In [3]:
#%% Set variables

#Directory containing participant data downloaded from gnacek.com/affective-video-database-online-study
data_directory = (r"../data")
# Identifier for the notebook when creating temp files
NOTEBOOK_NAME = "0_verify/" 

#Bool variables to enable/disable processing of segments of the data to speed up the runtime
process_slow_movement = False
process_fast_movement = False
process_video = True

validate_data = True

calculate_duration = True
calculate_average_av_ratings = True

#Bool variables to enable/disable plotting of graphs
plots_output_root_folder = "./temp/"+NOTEBOOK_NAME
plot_slow_movement_ppg = False
plot_contact_fit_state = False

no_of_frames_to_drop_from_start_of_recording = 1001


In [4]:
# Data for each participant is stored within a folder. Names of files indicate which segment of remote study it is
list_data_dir = os.listdir(data_directory)
is_folder = [os.path.isdir(os.path.join(data_directory,d)) for d in list_data_dir]
participants_list = np.array(list_data_dir)[ is_folder ]

#%% Get list of participant folders/files
print("Found data folders for " + str(len(participants_list)) + " participants")

Found data folders for 37 participants


In [5]:
#%% Run data validation check on all participants. Here we verify the following:
# - Number of files (10 total per participant: 5 csv and 5 json files)
# - Minimum signal quality treshold was reached during calibration fit check (mask can fit participants face)
# - ALl expected events are present and are in correct order

#return: participant number, signal_quality at the end of fit check (0-9 - higher is better), protocol (v1 - fully remote, v2 - remote from seperate lab room)
if(validate_data):
    fit_states_signal_quality = None
    for participant in participants_list:
        new_df = verify_data(data_directory + "/" + participant)
        if (fit_states_signal_quality is None):
            fit_states_signal_quality = new_df.copy()
        else:
            fit_states_signal_quality = pd.concat([fit_states_signal_quality,new_df], axis=0, ignore_index=True)
    print("----FINISHED DATA CHECK FOR ALL PARTICIPANTS SEE ABOVE FOR OUTPUT----")

 Running data check for:  ../data/participant_101
 Running data check for:  ../data/participant_216
 Running data check for:  ../data/participant_219
 Running data check for:  ../data/participant_222
 Running data check for:  ../data/participant_247
 Running data check for:  ../data/participant_248
 Running data check for:  ../data/participant_268
 Running data check for:  ../data/participant_270
 Running data check for:  ../data/participant_278
 Running data check for:  ../data/participant_290
 Running data check for:  ../data/participant_293
 Running data check for:  ../data/participant_299
 Running data check for:  ../data/participant_307
 Running data check for:  ../data/participant_308
 Running data check for:  ../data/participant_309
 Running data check for:  ../data/participant_310
 Running data check for:  ../data/participant_312
 Running data check for:  ../data/participant_314
 Running data check for:  ../data/participant_321
 Running data check for:  ../data/participant_322


In [6]:
fit_states_signal_quality

,participant_number,video_signal_quality,protocol
0,101,9,v1
1,216,9,v1
2,219,9,v1
3,222,9,v1
4,247,9,v1
5,248,9,v1
6,268,9,v1
7,270,9,v1
8,278,9,v1
9,290,9,v1


In [7]:
# Save
fit_states_signal_quality.to_csv( generate_complete_path("fit_states_signal_quality", subfolders=NOTEBOOK_NAME, file_extension=".csv") )

In [8]:
def drop_start_frames(data, frames_to_drop):
    return data.drop(data.index[0:frames_to_drop])

In [9]:
#%% Main analysis script

if(calculate_duration):
    durations = pd.DataFrame(columns = get_durations_columns())
if(calculate_average_av_ratings):
     average_av_ratings = pd.DataFrame(columns = get_average_av_ratings_columns())

participant_counter = 1

#Set it to participant index (0-38) if you wish to start at specific participant
custom_start_index = -1

#Loop through all participant data
for participant in participants_list:
    
    #Run for specific participant
    if(custom_start_index!=-1):
        if(participant_counter==1):
            participant = participants_list[custom_start_index]
        else:
            participant = participants_list[custom_start_index+participant_counter]
            
    ParticipantObj = Participant(participant, data_directory)

    print("Processing data for: " + ParticipantObj.name + ". " + str(participant_counter) + " out of " + str(len(participants_list)))
    ParticipantObj = skip_participant(ParticipantObj)
    
    if process_video:
        if(ParticipantObj.skip_video == False):
            print("Video Start")
            if(ParticipantObj.skip_video_1 == False):
                video_1_data = ParticipantObj.getVideo1Data()
                video_1_events = video_1_data[video_1_data['Event'] != '']
                video_1_data = drop_start_frames(video_1_data, no_of_frames_to_drop_from_start_of_recording)
                if(calculate_duration):
                    video_1_duration = round((video_1_data['Time'].iloc[-1] - video_1_data['Time'].iloc[0]),3)
                    video_1_GoodDuration, video_1_TimeTakenToEstablishGoodSignal = calculate_good_signal_quality_duration(video_1_data)
                    video_1_SignalCheckDuration = calculate_signal_quality_check_duration(video_1_events)
                    video_1_DataExcludingSignalCheck = video_1_data.drop(video_1_data.index[0:video_1_events['Frame#'].iloc[1]-1])
                    video_1_DurationExcludingSignalCheck = round((video_1_DataExcludingSignalCheck['Time'].iloc[-1] - video_1_DataExcludingSignalCheck['Time'].iloc[0]),3)
                    video_1_GoodDurationExcludingSignalCheck, blank = calculate_good_signal_quality_duration(video_1_data.drop(video_1_data.index[0:video_1_events['Frame#'].iloc[1]-1])) #drop 
                if plot_contact_fit_state:
                    plot_fit_state(video_1_data, video_1_events, ParticipantObj.name, "Video 1", root_folder=plots_output_root_folder)
            else:
                print("Skipping VIDEO_1 for:" + ParticipantObj.name)
                video_1_duration = "SKIPPED" 
                video_1_GoodDuration = "SKIPPED"  
                video_1_TimeTakenToEstablishGoodSignal = "SKIPPED" 
                video_1_SignalCheckDuration = "SKIPPED" 
                video_1_DurationExcludingSignalCheck = "SKIPPED" 
                video_1_GoodDurationExcludingSignalCheck = "SKIPPED"
                
            if(ParticipantObj.skip_video_2 == False):
                video_2_data = ParticipantObj.getVideo2Data()
                video_2_events = video_2_data[video_2_data['Event'] != '']
                if(calculate_average_av_ratings):
                    video_2_ratings = calculate_average_arousal_valence_ratings(video_2_events)
                video_2_data = drop_start_frames(video_2_data, no_of_frames_to_drop_from_start_of_recording)
                if(calculate_duration):
                    video_2_duration = video_2_data['Time'].iloc[-1] - video_2_data['Time'].iloc[0]
                    video_2_GoodDuration, blank = calculate_good_signal_quality_duration(video_2_data)
                if plot_contact_fit_state:
                    plot_fit_state(video_2_data, video_2_events, ParticipantObj.name, "Video 2", root_folder=plots_output_root_folder)
            else:
                print("Skipping VIDEO_2 for:" + ParticipantObj.name)
                video_2_duration = "SKIPPED" 
                video_2_GoodDuration = "SKIPPED" 
                
            if(ParticipantObj.skip_video_3 == False):
                video_3_data = ParticipantObj.getVideo3Data()
                video_3_events = video_3_data[video_3_data['Event'] != '']
                if(calculate_average_av_ratings):
                    video_3_ratings = calculate_average_arousal_valence_ratings(video_3_events)
                video_3_data = drop_start_frames(video_3_data, no_of_frames_to_drop_from_start_of_recording)
                if(calculate_duration):
                    video_3_duration = video_3_data['Time'].iloc[-1] - video_3_data['Time'].iloc[0]
                    video_3_GoodDuration, blank = calculate_good_signal_quality_duration(video_3_data)
                if plot_contact_fit_state:
                    plot_fit_state(video_3_data, video_3_events, ParticipantObj.name, "Video 3", root_folder=plots_output_root_folder)
            else:
                print("Skipping VIDEO_3 for:" + ParticipantObj.name)
                video_3_duration = "SKIPPED" 
                video_3_GoodDuration = "SKIPPED" 
                
            if(ParticipantObj.skip_video_4 == False):
                video_4_data = ParticipantObj.getVideo4Data()
                video_4_events = video_4_data[video_4_data['Event'] != '']
                if(calculate_average_av_ratings):
                    video_4_ratings = calculate_average_arousal_valence_ratings(video_4_events)
                video_4_data = drop_start_frames(video_4_data, no_of_frames_to_drop_from_start_of_recording)
                if(calculate_duration):
                    video_4_duration = video_4_data['Time'].iloc[-1] - video_4_data['Time'].iloc[0]
                    video_4_GoodDuration, blank = calculate_good_signal_quality_duration(video_4_data)
                if plot_contact_fit_state:
                    plot_fit_state(video_4_data, video_4_events, ParticipantObj.name, "Video 4", root_folder=plots_output_root_folder)
            else:
                print("Skipping VIDEO_4 for:" + ParticipantObj.name)
                video_4_duration = "SKIPPED" 
                video_4_GoodDuration = "SKIPPED" 

            if(ParticipantObj.skip_video_5 == False):
                video_5_data = ParticipantObj.getVideo5Data()
                video_5_events = video_5_data[video_5_data['Event'] != '']
                if(calculate_average_av_ratings):
                    video_5_ratings = calculate_average_arousal_valence_ratings(video_5_events)
                video_5_data = drop_start_frames(video_5_data, no_of_frames_to_drop_from_start_of_recording)
                if(calculate_duration):
                    video_5_duration = video_5_data['Time'].iloc[-1] - video_5_data['Time'].iloc[0]
                    video_5_GoodDuration, blank = calculate_good_signal_quality_duration(video_5_data)
                if plot_contact_fit_state:
                    plot_fit_state(video_5_data, video_5_events, ParticipantObj.name, "Video 5", root_folder=plots_output_root_folder)
            else:
                print("Skipping VIDEO_5 for:" + ParticipantObj.name)
                video_5_duration = "SKIPPED" 
                video_5_GoodDuration = "SKIPPED" 
            
            if(calculate_average_av_ratings):
                current_participant_average_av_ratings = {"participant": ParticipantObj.name, 
                                      "relax_2_valence": None, "relax_2_arousal": None, "relax_2_no_ratings": None, "relax_3_valence": None, "relax_3_arousal": None, "relax_3_no_ratings": None, "relax_4_valence": None, "relax_4_arousal": None, "relax_4_no_ratings": None, "relax_5_valence": None, "relax_5_arousal": None, "relax_5_no_ratings": None,
                                      "video_03_valence": None, "video_03_arousal": None, "video_03_no_ratings": None, "video_04_valence": None, "video_04_arousal": None, "video_04_no_ratings": None, "video_05_valence": None, "video_05_arousal": None, "video_05_no_ratings": None, "video_06_valence": None, "video_06_arousal": None, "video_06_no_ratings": None, "video_10_valence": None, "video_10_arousal": None, "video_10_no_ratings": None, "video_12_valence": None, "video_12_arousal": None, "video_12_no_ratings": None, "video_13_valence": None, "video_13_arousal": None, "video_13_no_ratings": None, "video_18_valence": None, "video_18_arousal": None, "video_18_no_ratings": None, "video_19_valence": None, "video_19_arousal": None, "video_19_no_ratings": None, "video_20_valence": None, "video_20_arousal": None, "video_20_no_ratings": None,
                                      "video_21_valence": None, "video_21_arousal": None, "video_21_no_ratings": None, "video_22_valence": None, "video_22_arousal": None, "video_22_no_ratings": None, "video_23_valence": None, "video_23_arousal": None, "video_23_no_ratings": None, "video_25_valence": None, "video_25_arousal": None, "video_25_no_ratings": None, "video_29_valence": None, "video_29_arousal": None, "video_29_no_ratings": None, "video_31_valence": None, "video_31_arousal": None, "video_31_no_ratings": None, "video_33_valence": None, "video_33_arousal": None, "video_33_no_ratings": None, "video_37_valence": None, "video_37_arousal": None, "video_37_no_ratings": None, "video_38_valence": None, "video_38_arousal": None, "video_38_no_ratings": None, "video_39_valence": None, "video_39_arousal": None, "video_39_no_ratings": None,
                                      "video_41_valence": None, "video_41_arousal": None, "video_41_no_ratings": None, "video_42_valence": None, "video_42_arousal": None, "video_42_no_ratings": None, "video_46_valence": None, "video_46_arousal": None, "video_46_no_ratings": None, "video_48_valence": None, "video_48_arousal": None, "video_48_no_ratings": None, "video_49_valence": None, "video_49_arousal": None, "video_49_no_ratings": None, "video_51_valence": None, "video_51_arousal": None, "video_51_no_ratings": None, "video_55_valence": None, "video_55_arousal": None, "video_55_no_ratings": None, "video_56_valence": None, "video_56_arousal": None, "video_56_no_ratings": None, "video_57_valence": None, "video_57_arousal": None, "video_57_no_ratings": None, "video_58_valence": None, "video_58_arousal": None, "video_58_no_ratings": None}  
                current_participant_average_av_ratings['relax_2_valence'] = video_2_ratings[0].average_valence
                current_participant_average_av_ratings['relax_2_arousal'] = video_2_ratings[0].average_arousal
                current_participant_average_av_ratings['relax_2_no_ratings'] = len(video_2_ratings[0].arousal_ratings)
                current_participant_average_av_ratings['relax_3_valence'] = video_3_ratings[0].average_valence
                current_participant_average_av_ratings['relax_3_arousal'] = video_3_ratings[0].average_arousal
                current_participant_average_av_ratings['relax_3_no_ratings'] = len(video_3_ratings[0].arousal_ratings)
                current_participant_average_av_ratings['relax_4_valence'] = video_4_ratings[0].average_valence
                current_participant_average_av_ratings['relax_4_arousal'] = video_4_ratings[0].average_arousal
                current_participant_average_av_ratings['relax_4_no_ratings'] = len(video_4_ratings[0].arousal_ratings)
                current_participant_average_av_ratings['relax_5_valence'] = video_5_ratings[0].average_valence
                current_participant_average_av_ratings['relax_5_arousal'] = video_5_ratings[0].average_arousal
                current_participant_average_av_ratings['relax_5_no_ratings'] = len(video_5_ratings[0].arousal_ratings)
                
                combined_affective_video_ratings = video_2_ratings + video_3_ratings + video_4_ratings
                for video_rating in combined_affective_video_ratings:
                    if(video_rating.name!="relax"):
                        current_participant_average_av_ratings["video_" + video_rating.name + "_valence"] = video_rating.average_valence
                        current_participant_average_av_ratings["video_" + video_rating.name + "_arousal"] = video_rating.average_arousal
                        current_participant_average_av_ratings["video_" + video_rating.name + "_no_ratings"] = len(video_rating.arousal_ratings)

                average_av_ratings = pd.concat([average_av_ratings, pd.DataFrame.from_dict([current_participant_average_av_ratings])], ignore_index = True)
            print("Video End")
        else:
            print("Skipping VIDEO for:" + ParticipantObj.name)
            video_1_duration = "SKIPPED" 
            video_1_GoodDuration = "SKIPPED"  
            video_1_TimeTakenToEstablishGoodSignal = "SKIPPED" 
            video_1_SignalCheckDuration = "SKIPPED" 
            video_1_DurationExcludingSignalCheck = "SKIPPED" 
            video_1_GoodDurationExcludingSignalCheck = "SKIPPED" 
            video_2_duration = "SKIPPED" 
            video_2_GoodDuration = "SKIPPED" 
            video_3_duration = "SKIPPED" 
            video_3_GoodDuration = "SKIPPED" 
            video_4_duration = "SKIPPED" 
            video_4_GoodDuration = "SKIPPED" 
            video_5_duration = "SKIPPED" 
            video_5_GoodDuration = "SKIPPED" 
    
    if(calculate_duration):
        current_participant_durations = {'participant': ParticipantObj.name,
                                         'video_1_total_duration': video_1_duration,'video_1_total_good_fit': video_1_GoodDuration, 'video_1_TimeTakenToEstablishGoodSignal': video_1_TimeTakenToEstablishGoodSignal,'video_1_signal_check_duration': video_1_SignalCheckDuration,'video_1_duration_excluding_signal_check': video_1_DurationExcludingSignalCheck,'video_1_good_fit_excluding_signal_check': video_1_GoodDurationExcludingSignalCheck,
                                         'video_2_duration': video_2_duration, 'video_2_good_fit': video_2_GoodDuration,
                                         'video_3_duration': video_3_duration, 'video_3_good_fit': video_3_GoodDuration,
                                         'video_4_duration': video_4_duration, 'video_4_good_fit': video_4_GoodDuration,
                                         'video_5_duration': video_5_duration, 'video_5_good_fit': video_5_GoodDuration}

        durations = pd.concat([durations,pd.DataFrame.from_dict([current_participant_durations])], ignore_index = True)
   
    participant_counter = participant_counter + 1
print("Finished processing data for: " + ParticipantObj.name)

Processing data for: participant_101. 1 out of 37
Video Start
Video End
Processing data for: participant_216. 2 out of 37
Video Start
Video End
Processing data for: participant_219. 3 out of 37
Video Start
Video End
Processing data for: participant_222. 4 out of 37
Video Start
Video End
Processing data for: participant_247. 5 out of 37
Video Start
Video End
Processing data for: participant_248. 6 out of 37
Video Start
Video End
Processing data for: participant_268. 7 out of 37
Video Start
Video End
Processing data for: participant_270. 8 out of 37
Video Start
Video End
Processing data for: participant_278. 9 out of 37
Video Start
Video End
Processing data for: participant_290. 10 out of 37
Video Start
Video End
Processing data for: participant_293. 11 out of 37
Video Start
Video End
Processing data for: participant_299. 12 out of 37
Video Start
Video End
Processing data for: participant_307. 13 out of 37
Video Start
Video End
Processing data for: participant_308. 14 out of 37
Video Sta

In [10]:
average_av_ratings

,participant,relax_2_valence,relax_2_arousal,relax_2_no_ratings,relax_3_valence,relax_3_arousal,relax_3_no_ratings,relax_4_valence,relax_4_arousal,relax_4_no_ratings,relax_5_valence,relax_5_arousal,relax_5_no_ratings,video_03_valence,video_03_arousal,video_03_no_ratings,video_04_valence,video_04_arousal,video_04_no_ratings,video_05_valence,video_05_arousal,video_05_no_ratings,video_06_valence,video_06_arousal,video_06_no_ratings,video_10_valence,video_10_arousal,video_10_no_ratings,video_12_valence,video_12_arousal,video_12_no_ratings,video_13_valence,video_13_arousal,video_13_no_ratings,video_18_valence,video_18_arousal,video_18_no_ratings,video_19_valence,video_19_arousal,video_19_no_ratings,...,video_33_no_ratings,video_37_valence,video_37_arousal,video_37_no_ratings,video_38_valence,video_38_arousal,video_38_no_ratings,video_39_valence,video_39_arousal,video_39_no_ratings,video_41_valence,video_41_arousal,video_41_no_ratings,video_42_valence,video_42_arousal,video_42_no_ratings,video_46_valence,video_46_arousal,video_46_no_ratings,video_48_valence,video_48_arousal,video_48_no_ratings,video_49_valence,video_49_arousal,video_49_no_ratings,video_51_valence,video_51_arousal,video_51_no_ratings,video_55_valence,video_55_arousal,video_55_no_ratings,video_56_valence,video_56_arousal,video_56_no_ratings,video_57_valence,video_57_arousal,video_57_no_ratings,video_58_valence,video_58_arousal,video_58_no_ratings
0,participant_101,6.633,3.041,49,4.950,2.500,40,7.585,5.780,41,3.733,1.000,15,1.000,8.300,10,1.000,8.182,11,1.111,6.889,9,2.588,6.882,17,1.556,7.889,9,1.000,8.167,6,1.000,8.000,9,1.692,7.692,13,1.000,8.500,6,...,3,5.714,4.286,7,6.100,4.200,10,4.000,5.750,12,7.667,5.000,9,7.000,6.100,10,6.800,6.900,10,8.273,5.364,11,7.400,7.600,20,4.667,6.048,21,6.692,8.000,13,5.000,9.000,8,5.000,8.538,13,6.389,6.444,18
1,participant_216,4.847,5.102,59,4.476,3.667,21,3.057,7.057,35,5.069,5.317,145,3.057,7.057,35,3.216,7.703,37,3.059,5.941,34,5.139,7.333,36,1.794,6.912,34,2.545,6.745,55,3.026,7.282,39,1.926,5.444,27,1.944,5.556,36,...,39,5.949,6.205,39,4.522,3.261,23,4.531,4.906,32,6.160,6.680,25,6.033,6.667,30,5.321,6.536,28,5.593,7.519,27,6.844,7.125,32,7.150,6.950,40,4.847,5.102,59,4.074,5.519,27,4.167,6.750,48,6.465,7.419,43
2,participant_219,6.226,1.321,53,4.804,3.435,46,4.950,3.400,20,5.000,2.267,15,3.250,7.500,24,2.625,7.750,16,3.593,7.593,27,3.533,7.267,15,3.469,7.673,49,4.892,7.892,37,2.867,8.333,15,5.907,7.372,43,4.531,7.688,32,...,18,7.545,5.545,22,5.810,2.333,21,4.545,6.485,33,8.091,6.818,11,6.412,6.794,34,6.571,6.179,28,7.152,6.303,33,7.821,6.857,28,6.042,6.875,24,3.531,6.184,49,6.250,6.450,20,7.056,6.500,18,7.038,6.308,26
3,participant_222,4.387,2.742,31,5.741,4.296,27,4.647,2.706,17,3.818,1.455,11,4.591,7.364,22,1.000,8.500,2,3.211,7.132,38,2.800,6.600,5,5.140,8.140,50,2.625,8.375,32,2.500,7.250,12,2.650,7.450,20,2.880,7.560,25,...,21,5.895,6.579,19,4.500,4.400,10,6.851,7.104,67,8.275,8.050,40,6.545,7.818,22,6.711,6.868,38,6.310,6.190,42,6.822,7.022,45,8.276,6.862,29,8.214,7.429,14,6.537,6.902,41,6.000,5.722,18,7.500,7.364,22
4,participant_247,6.612,4.490,49,6.583,4.167,24,4.837,2.047,43,4.581,3.645,31,3.455,7.500,22,3.778,6.389,18,3.032,6.677,31,3.407,4.407,27,3.211,6.895,38,5.227,8.182,22,2.345,5.862,29,3.042,4.833,24,2.750,3.750,4,...,27,6.583,4.167,24,5.511,3.356,45,7.269,5.769,26,7.074,4.037,27,5.071,5.143,28,5.400,6.167,30,4.826,5.478,23,6.917,5.528,36,5.074,4.926,27,7.000,6.938,16,6.147,6.765,34,5.167,3.667,18,5.172,6.379,29
5,participant_248,6.512,2.442,86,7.148,2.025,81,4.103,5.717,145,6.559,2.255,102,2.385,7.128,39,2.312,6.250,32,2.159,3.058,69,3.922,4.882,51,2.771,7.354,48,3.952,5.976,42,2.741,7.370,27,2.694,5.878,49,1.765,6.294,17,...,24,6.862,3.276,29,6.026,4.000,39,6.303,2.879,33,5.674,4.837,43,6.333,4.667,30,5.390,5.659,41,6.962,5.500,26,6.517,4.931,29,5.724,5.276,29,7.500,4.125,32,5.833,5.111,18,5.448,5.000,29,6.632,5.737,38
6,participant_268,7.694,2.194,36,1.844,5.000,64,3.886,3.955,44,6.525,5.390,177,2.364,6.636

In [11]:
durations

,participant,video_1_total_duration,video_1_total_good_fit,video_1_TimeTakenToEstablishGoodSignal,video_1_signal_check_duration,video_1_duration_excluding_signal_check,video_1_good_fit_excluding_signal_check,video_2_duration,video_2_good_fit,video_3_duration,video_3_good_fit,video_4_duration,video_4_good_fit,video_5_duration,video_5_good_fit
0,participant_101,174.933,174.933,0.000,13.793,159.508,159.508,426.25134,425.139,426.38598,423.228,426.42993,424.198,122.51307,121.426
1,participant_216,148.338,147.279,1.059,6.613,141.725,141.725,418.50501,417.446,418.44604,417.381,418.46900,418.469,118.60803,115.455
2,participant_219,291.055,289.985,1.070,10.000,266.723,266.723,424.90771,423.841,425.43994,424.375,425.76514,424.687,121.18262,120.113
3,participant_222,230.440,227.682,1.071,9.320,219.959,218.272,424.13098,423.062,425.07410,421.731,425.68176,423.303,122.38782,121.321
4,participant_247,268.962,267.905,1.057,6.373,262.590,262.590,422.90943,421.856,423.25940,422.209,423.52722,413.585,123.96949,117.784
5,participant_248,363.180,362.095,1.085,11.209,350.044,350.044,426.16125,425.052,426.33398,425.221,426.51599,423.350,122.51147,119.307
6,participant_268,252.946,39.635,1.058,17.493,234.107,21.854,425.47461,321.988,425.74085,414.851,426.03382,303.129,122.42652,5.261
7,participant_270,375.826,368.364,1.090,33.193,340.704,340.704,425.71912,424.641,426.18066,425.111,426.47302,423.312,122.56750,121.494
8,participant_278,247.176,246.095,1.081,11.524,234.431,234.431,426.58337,163.119,426.69190,390.302,426.73206,422.115,122.63293,101.655
9,participant_290,223.312,222.244,1.067,23.732,193.616,193.616,426.18811,425.110,426.53271,421.287,426.80371,425.699,122.65356,121.576


In [12]:
# Save result files
average_av_ratings.to_csv( generate_complete_path("average_av_ratings", subfolders=NOTEBOOK_NAME, file_extension=".csv") )
durations.to_csv( generate_complete_path("durations", subfolders=NOTEBOOK_NAME, file_extension=".csv") )

In [13]:
# Durations
import datetime
#Total duration
total_duration_seconds = (durations['video_1_total_duration'].sum() + durations['video_2_duration'].sum() + 
                          durations['video_3_duration'].sum() + durations['video_4_duration'].sum() + durations['video_5_duration'].sum())

mean_total_duration_seconds = (durations['video_1_total_duration'].mean() + durations['video_2_duration'].mean() + 
                          durations['video_3_duration'].mean() + durations['video_4_duration'].mean() + durations['video_5_duration'].mean())

min_total_duration_seconds = (durations['video_1_total_duration'].min() + durations['video_2_duration'].min() + 
                          durations['video_3_duration'].min() + durations['video_4_duration'].min() + durations['video_5_duration'].min())

max_total_duration_seconds = (durations['video_1_total_duration'].max() + durations['video_2_duration'].max() + 
                          durations['video_3_duration'].max() + durations['video_4_duration'].max() + durations['video_5_duration'].max())

print('Total Duration: ' + str(datetime.timedelta(seconds=total_duration_seconds)))
print('Mean Total Duration: ' + str(datetime.timedelta(seconds=mean_total_duration_seconds)))
print('Min Total Duration: ' + str(datetime.timedelta(seconds=min_total_duration_seconds)))
print('Max Total Duration: ' + str(datetime.timedelta(seconds=max_total_duration_seconds)))

#Total affective duration (good_fit)
total_affective_duration_good_fit_seconds = (durations['video_2_good_fit'].sum() + durations['video_3_good_fit'].sum() + 
                          durations['video_4_good_fit'].sum() + durations['video_5_good_fit'].sum())

mean_affective_duration_good_fit_seconds = (durations['video_2_good_fit'].mean() + durations['video_3_good_fit'].mean() + 
                          durations['video_4_good_fit'].mean() + durations['video_5_good_fit'].mean())

min_affective_duration_good_fit_seconds = (durations['video_2_good_fit'].min() + durations['video_3_good_fit'].min() + 
                          durations['video_4_good_fit'].min() + durations['video_5_good_fit'].min())

max_affective_duration_good_fit_seconds = (durations['video_2_good_fit'].max() + durations['video_3_good_fit'].max() + 
                          durations['video_4_good_fit'].max() + durations['video_5_good_fit'].max())


print('\nTotal affective duration (good_fit): ' + str(datetime.timedelta(seconds=total_affective_duration_good_fit_seconds)))
print('Mean affective duration (good_fit): ' + str(datetime.timedelta(seconds=mean_affective_duration_good_fit_seconds)))
print('Min affective duration (good_fit): ' + str(datetime.timedelta(seconds=min_affective_duration_good_fit_seconds)))
print('Max affective duration (good_fit): ' + str(datetime.timedelta(seconds=max_affective_duration_good_fit_seconds)))


Total Duration: 17:13:15.983810
Mean Total Duration: 0:27:55.567130
Min Total Duration: 0:25:22.270730
Max Total Duration: 0:32:24.744920

Total affective duration (good_fit): 14:01:57.982000
Mean affective duration (good_fit): 0:22:45.350865
Min affective duration (good_fit): 0:14:19.651000
Max affective duration (good_fit): 0:23:19.676000
